In [27]:
import importlib
import JobsMapResultsFilesToContainerObjs as ImageMap
import pandas as pd
import statistics as s
import re

gidAidMapFl = "../data/experiment2_gid_aid_map.json"
aidFeatureMapFl = "../data/experiment2_aid_features.json"
imgJobMap = "../data/imageGID_job_map_expt2_corrected.csv"
importlib.reload(ImageMap)

<module 'JobsMapResultsFilesToContainerObjs' from '/Users/sreejithmenon/Google Drive/PythonCode/AnimalPhotoBias/script/JobsMapResultsFilesToContainerObjs.py'>

In [34]:
'''
resultsPerJobDf > Gives you shares/not shares per image per album (Python Object of .results file converted to DF)
resultsPerJobDf['GID','Album','Shared','Not Shared','Proportion']
'''
imgAlbumDict = ImageMap.genImgAlbumDictFromMap("../data/imageGID_job_map_expt2_corrected.csv")
master = ImageMap.createResultDict(1,100)
imgShareNotShareList,noResponse = ImageMap.imgShareCountsPerAlbum(imgAlbumDict,master)

resultsPerJobDf = pd.DataFrame(imgShareNotShareList,columns = ['GID','Album','Shared','Not Shared','Proportion'])

'''
Code for reading from json files into data frames
aidGidDf['AID','GID']
aidFeaturesDf['AID',[FEATURES]]
'''
aidGidDict = ImageMap.genAidGidTupListFromMap('../data/experiment2_gid_aid_map.json')
aidGidDf= pd.DataFrame(aidGidDict,columns = ['AID','GID'])

aidFeaturesDf = pd.DataFrame(ImageMap.genAidFeatureDictList('../data/experiment2_aid_features.json'))
aidFeaturesDf['AID'] = aidFeaturesDf['AID'].astype('int32')

'''
rankListImgsDf  > Gives you the results of number of times each image was shared overall
rankListImgsDf['GID','Shared','Not Shared','Proportion']
'''
rankListImgsDf = resultsPerJobDf.groupby(['GID'])['Shared','Not Shared'].sum() 
rankListImgsDf['Total'] = rankListImgsDf['Shared'] + rankListImgsDf['Not Shared']
rankListImgsDf['Proportion'] = rankListImgsDf['Shared'] * 100 / rankListImgsDf['Total']
rankListImgsDf = rankListImgsDf.sort_values(by = ['Proportion'],ascending = False)
rankListImgsDf.to_csv('../data/rankListImages.csv')

'''
resultsAIDGIDDf > Merged data frame that add's AID info to the results data
resultsAIDGIDDf['AID' + [resultsPerJobDf]]

gidAidResultsFeaturesDf > A master data frame that has results data merged along with all the image features
gidAidResultsFeaturesDf['GID','AID',[FEATURES],[resultsPerJobDf]]

'''
resultsAIDGIDDf = pd.merge(aidGidDf,resultsPerJobDf,left_on='GID',right_on = 'GID',how="right")

gidAidResultsFeaturesDf = pd.merge(resultsAIDGIDDf,aidFeaturesDf,left_on = 'AID',right_on = 'AID') # most important data frame with all the info
gidAidResultsFeaturesDf.to_csv("../data/resultsFeaturesComb.csv",index=False)

In [53]:
auditResMap(imgAlbumDict,imgShareNotShareList)

['photo_album_22']
['photo_album_22']
['photo_album_86', 'photo_album_4', 'photo_album_91']
['photo_album_86', 'photo_album_4', 'photo_album_91']
['photo_album_93']
['photo_album_93']
['photo_album_2']
['photo_album_2']
['photo_album_95']
['photo_album_95']
['photo_album_84']
['photo_album_84']
['photo_album_23', 'photo_album_68']
['photo_album_23', 'photo_album_68']
['photo_album_41']
['photo_album_41']
['photo_album_98', 'photo_album_61']
['photo_album_98', 'photo_album_61']
['photo_album_58', 'photo_album_51', 'photo_album_77']
['photo_album_58', 'photo_album_51', 'photo_album_77']
['photo_album_77']
['photo_album_77']
['photo_album_39']
['photo_album_39']
['photo_album_96']
['photo_album_96']
['photo_album_42', 'photo_album_17', 'photo_album_55', 'photo_album_67']
['photo_album_42', 'photo_album_17', 'photo_album_55', 'photo_album_67']
['photo_album_74']
['photo_album_74']
['photo_album_32']
['photo_album_32']
['photo_album_69']
['photo_album_69']
['photo_album_52']
['photo_album_5

(False, False, False)

In [72]:
#gen imgAlbmDict from results:
imgAlbumDict = ImageMap.genImgAlbumDictFromMap("../data/imageGID_job_map_expt2_corrected.csv")
master = ImageMap.createResultDict(1,100)
imgShareNotShareList,noResponse = ImageMap.imgShareCountsPerAlbum(imgAlbumDict,master)

auditResMap(imgAlbumDict,imgShareNotShareList)



Error 2:  1 photo_album_57
Error 2:  1 photo_album_85


(False, True, False)

In [71]:
def auditResMap(imgAlbumDict,resultList):
    err1 = False
    err2 = False
    err3 = False
    imgAlbmDctRest = {}
    
    for tup in resultList:
        imgAlbmDctRest[tup[0]] = imgAlbmDctRest.get(tup[0],[]) + [tup[1]]
            
    for tup in resultList:
        shouldBeInAlbms = imgAlbumDict[tup[0]]
        
        if tup[1] not in shouldBeInAlbms:
            print("Error1: " ,tup[1])
            err1 = True

    
    for gid in imgAlbumDict.keys():
        albmRes = imgAlbmDctRest.get(gid,[])
        albmOri = imgAlbumDict.get(gid,[])
        for albm in albmOri:
            if albm not in albmRes:
                print("Error 2: ",gid,albm)
                err2 = True
        
        for albm in albmRes:
            if albm not in albmOri:
                print("Error 3: ",gid,albm)
                err3 = True
      
    return err1,err2,err3

In [59]:
imgAlbumDict['1']

['photo_album_57', 'photo_album_85']

In [60]:
master

{'photo_album_1': {'Answer.350': ['350|noShare',
   '350|noShare',
   '350|noShare',
   '350|share',
   '350|share',
   '350|noShare',
   '350|noShare',
   '350|noShare',
   '350|noShare',
   '350|noShare'],
  'Answer.351': ['351|share',
   '351|share',
   '351|share',
   '351|share',
   '351|share',
   '351|share',
   '351|share',
   '351|share',
   '351|share',
   '351|share'],
  'Answer.352': ['352',
   '352|noShare',
   '352|noShare',
   '352|noShare',
   '352|noShare',
   '352|noShare',
   '352|noShare',
   '352|noShare',
   '352|noShare',
   '352|share'],
  'Answer.353': ['353|share',
   '353|noShare',
   '353|share',
   '353|noShare',
   '353|share',
   '353|noShare',
   '353|share',
   '353|share',
   '353|share',
   '353|noShare'],
  'Answer.354': ['354|share',
   '354|share',
   '354|share',
   '354|share',
   '354|share',
   '354|noShare',
   '354|share',
   '354|share',
   '354|share',
   '354|noShare'],
  'Answer.359': ['359|noShare',
   '359|noShare',
   '359|noShare',
  